In [1]:
!python /home/jovyan/Documents/workspace/TemporalTransformer/setup.py develop

running develop
running egg_info
writing TemporalTransformer.egg-info/PKG-INFO
writing dependency_links to TemporalTransformer.egg-info/dependency_links.txt
writing top-level names to TemporalTransformer.egg-info/top_level.txt
reading manifest file 'TemporalTransformer.egg-info/SOURCES.txt'
writing manifest file 'TemporalTransformer.egg-info/SOURCES.txt'
running build_ext
Creating /opt/conda/lib/python3.7/site-packages/TemporalTransformer.egg-link (link to .)
TemporalTransformer 0.0.1 is already the active version in easy-install.pth

Installed /home/jovyan/Documents/workspace/TemporalTransformer/TemporalTransformer/notebooks
Processing dependencies for TemporalTransformer==0.0.1
Finished processing dependencies for TemporalTransformer==0.0.1


In [2]:
from TemporalTransformer import Hopper
from TemporalTransformer import Prepper
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import numpy as np

# Hopper

In [3]:
verbose = True
h = Hopper.dbms(verbose=verbose)

DROP TABLE IF EXISTS i_ids;

CREATE TABLE i_ids (
	i_id TEXT NOT NULL,
	PRIMARY KEY (i_id)
);

DROP TABLE IF EXISTS i_windows;

CREATE TABLE i_windows (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_partitions;

CREATE TABLE i_partitions (
	i_id TEXT NOT NULL,
	partition TEXT NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_relcal;

CREATE TABLE i_relcal (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL
);



## Load Tables

In [4]:
_tc = Hopper.table_config("characteristics_0", 
                  ["age", "sex", "eth", "bin_ldc", "job"], 
                  ["real", "bin", "ldc", "ldc", "hdc"],
                  has_times=False,
                  primary_key=True)
h.create_fvm_with_csv(_tc, "data/characteristics_0.csv", delimiter=',')


_tc = Hopper.table_config("samples_0", 
                  ["SBP", "DBP", "type"], 
                  ["real", "real", "ldc"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_0.csv", delimiter=',')



_tc = Hopper.table_config("samples_1", 
                  ["tmps", "hrs", "sbps", "rrs", "dbps", "sats", "wgts"],
                  ["real", "real", 'real', "real", "real", "real", "real"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_1.csv", delimiter=',')


_tc = Hopper.table_config("samples_2", 
                  ["dx"],
                  ["hdc"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_2.csv", delimiter=',')



_tc = Hopper.table_config("samples_3", 
                  ["outcome"],
                  ["bin"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_3.csv", delimiter=',')


_tc = Hopper.table_config("samples_4", 
                  ["q"],
                  ["bin"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_4.csv", delimiter=',')


DROP TABLE IF EXISTS characteristics_0;

CREATE TABLE characteristics_0 (
	i_id TEXT NOT NULL,
	age REAL NOT NULL,
	sex TEXT NOT NULL,
	eth TEXT NOT NULL,
	bin_ldc TEXT NOT NULL,
	job TEXT NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

INSERT OR IGNORE INTO i_ids (i_id) VALUES (?);

INSERT  INTO characteristics_0 (i_id, age, sex, eth, bin_ldc, job) VALUES (?, ?, ?, ?, ?, ?);

CREATE INDEX i_index_characteristics_0_i_id ON characteristics_0(i_id);

CREATE INDEX i_index_characteristics_0_age ON characteristics_0(age);

CREATE INDEX i_index_characteristics_0_sex ON characteristics_0(sex);

CREATE INDEX i_index_characteristics_0_eth ON characteristics_0(eth);

CREATE INDEX i_index_characteristics_0_bin_ldc ON characteristics_0(bin_ldc);

CREATE INDEX i_index_characteristics_0_job ON characteristics_0(job);

10000 rows loaded

DROP TABLE IF EXISTS samples_0;

CREATE TABLE samples_0 (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL,
	SBP RE

In [5]:
h.dew_it(fit_normalization_via_sql_qds=False,
         default_first=0,
         default_last=5)

INSERT  INTO i_windows (i_id, i_st, i_et) VALUES (?, ?, ?);

gen_range_table_sql called: af=None, bf=None
INSERT OR IGNORE INTO i_windows 
SELECT * FROM (
	SELECT i_id, 0 AS i_st, 5 AS i_et
	FROM (
	    SELECT i_id, i_st, i_et
		FROM samples_0
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_1
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_2
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_3
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_4
		
	)
	GROUP BY i_id
);

INSERT  INTO i_partitions (i_id, partition) VALUES (?, ?);

INSERT OR IGNORE INTO i_partitions 
SELECT * FROM (
	SELECT i_id, CASE 
		 WHEN r <= 0.8 THEN "train" 
		 WHEN r <= 0.9 THEN "dev" 
		 WHEN r <= 1.0 THEN "test" 
		 ELSE "test" 
	END
	FROM (SELECT i_id, ABS(RANDOM())/(9223372036854775807.0) AS r FROM i_ids)
);

CREATE VIEW win_samples_0 AS
    SELECT samples_0.i_id,
	MAX(samples_0.i_st, i_windows.i_st) AS i_st,
	MIN(samples_0.i_et, i_windows.i_et) AS i_et,
	SBP,
	DBP,
	type
	FROM (
	    samples_0 JOIN i_windo

SELECT AVG(sats) AS avg,
       SUM((sats-(SELECT AVG(sats) FROM i_partitions_train_win_samples_1))*
       (sats-(SELECT AVG(sats) FROM i_partitions_train_win_samples_1)))
       /(COUNT(sats)-1) AS var
FROM i_partitions_train_win_samples_1
;

SELECT AVG(wgts) AS avg,
       SUM((wgts-(SELECT AVG(wgts) FROM i_partitions_train_win_samples_1))*
       (wgts-(SELECT AVG(wgts) FROM i_partitions_train_win_samples_1)))
       /(COUNT(wgts)-1) AS var
FROM i_partitions_train_win_samples_1
;

SELECT COUNT(*) FROM i_partitions_train_win_samples_2;

SELECT dx, COUNT(*) AS n
FROM i_partitions_train_win_samples_2
GROUP BY dx
ORDER BY n DESC
;


SELECT COUNT(*) FROM i_partitions_train_win_samples_3;

SELECT outcome, COUNT(*) AS n
FROM i_partitions_train_win_samples_3
GROUP BY outcome
ORDER BY n DESC
;


SELECT COUNT(*) FROM i_partitions_train_win_samples_4;

SELECT q, COUNT(*) AS n
FROM i_partitions_train_win_samples_4
GROUP BY q
ORDER BY n DESC
;


SELECT min(i_st) AS i_st, max(i_et) AS i_et
FROM 

SELECT avg_tmps, min_tmps, max_tmps, avg_hrs, min_hrs, max_hrs, avg_sbps, min_sbps, max_sbps, avg_rrs, min_rrs, max_rrs, avg_dbps, min_dbps, max_dbps, avg_sats, min_sats, max_sats, avg_wgts, min_wgts, max_wgts, count
FROM agg_samples_1
;

SELECT count
FROM agg_samples_2
;

SELECT count
FROM agg_samples_3
;

SELECT count
FROM agg_samples_4
;

CREATE VIEW nrm_characteristics_0 AS
    SELECT i_id,
	(age-45.0359)/9.922157587440344 as age,
	sex,
	eth_e0,
	eth_e1,
	eth_e2,
	eth_e3,
	eth_e4,
	eth_e5,
	eth__OTHER_,
	bin_ldc_A,
	bin_ldc_B,
	bin_ldc__OTHER_,
	job
	FROM ohe_characteristics_0
	
;

CREATE VIEW nrm_samples_0 AS
    SELECT i_id,
	i_st,
	i_et,
	(avg_SBP-120.02991782870583)/4.415600233733819 as avg_SBP,
	(min_SBP-118.77190053455995)/4.913301352229715 as min_SBP,
	(max_SBP-121.29005451806029)/4.899438656085264 as max_SBP,
	(avg_DBP-80.01451139723319)/4.367123251760052 as avg_DBP,
	(min_DBP-78.7617440364533)/4.856046849497142 as min_DBP,
	(max_DBP-81.26823426024221)/4.85165039165944 as m

In [6]:
dbms_save_dict = vars(h)
dbms_save_dict

{'database': ':memory:',
 'verbose': True,
 'con': <sqlite3.Connection at 0x7efd40138e30>,
 'cur': <sqlite3.Cursor at 0x7efd40002ce0>,
 'cur_man': <TemporalTransformer.sqlite_utils.cursor_manager at 0x7efd8e575fd0>,
 'mn_tc': <TemporalTransformer.Hopper.table_config at 0x7efd40359ad0>,
 'mn_dtm': <TemporalTransformer.Hopper.data_table_manager at 0x7efd4089a710>,
 'wn_tc': <TemporalTransformer.Hopper.table_config at 0x7efd40860ad0>,
 'wn_dtm': <TemporalTransformer.Hopper.data_table_manager at 0x7efd40860290>,
 'pr_tc': <TemporalTransformer.Hopper.table_config at 0x7efd408246d0>,
 'pr_dtm': <TemporalTransformer.Hopper.data_table_manager at 0x7efdb804f5d0>,
 'rc_tc': <TemporalTransformer.Hopper.table_config at 0x7efdb804f250>,
 'rc_dtm': <TemporalTransformer.Hopper.data_table_manager at 0x7efdbbc9fa50>,
 'windows': True,
 'partitions': True,
 'relcal_set': True,
 'fvms': [<TemporalTransformer.Hopper.flow_view_manager at 0x7efd40935510>,
  <TemporalTransformer.Hopper.flow_view_manager at 0

In [7]:
for fvm in dbms_save_dict["fvms"]:
    print(fvm.tc.name)

characteristics_0
samples_0
samples_1
samples_2
samples_3
samples_4


In [8]:
for fvm in dbms_save_dict["fvms"]:
    print(fvm.nrm_config)

{'age': {'avg': 45.0359, 'count': 10000, 'var': 98.44921118999999, 'type': 'real'}, 'sex': {'type': 'bin'}, 'eth_e0': {'type': 'ldc'}, 'eth_e1': {'type': 'ldc'}, 'eth_e2': {'type': 'ldc'}, 'eth_e3': {'type': 'ldc'}, 'eth_e4': {'type': 'ldc'}, 'eth_e5': {'type': 'ldc'}, 'eth__OTHER_': {'type': 'ldc'}, 'bin_ldc_A': {'type': 'ldc'}, 'bin_ldc_B': {'type': 'ldc'}, 'bin_ldc__OTHER_': {'type': 'ldc'}, 'job': {'type': 'hdc'}}
{'avg_SBP': {'avg': 120.02991782870583, 'count': 29464, 'var': 19.497525424150158, 'type': 'real'}, 'min_SBP': {'avg': 118.77190053455995, 'count': 29464, 'var': 24.140530177822352, 'type': 'real'}, 'max_SBP': {'avg': 121.29005451806029, 'count': 29464, 'var': 24.00449914474258, 'type': 'real'}, 'avg_DBP': {'avg': 80.01451139723319, 'count': 29464, 'var': 19.07176549606329, 'type': 'real'}, 'min_DBP': {'avg': 78.7617440364533, 'count': 29464, 'var': 23.581191004511115, 'type': 'real'}, 'max_DBP': {'avg': 81.26823426024221, 'count': 29464, 'var': 23.5385115228892, 'type': 

In [9]:
fvm_save_dicts = []
for fvm in dbms_save_dict["fvms"]:
    
    fvm_tc_save_dict = fvm.tc._get_save_dict()
    fvm_save_dict = {"tc": fvm_tc_save_dict,
                     "filter_config": fvm.filter_config,
                     "nrm_config": fvm.nrm_config}
    
    fvm_save_dicts.append(fvm_save_dict)
    
    #break

In [10]:
fvm_save_dicts

[{'tc': {'name': 'characteristics_0',
   'feature_names': ['age', 'sex', 'eth', 'bin_ldc', 'job'],
   'feature_types': ['real', 'bin', 'ldc', 'ldc', 'hdc'],
   'has_times': False,
   'primary_key': True,
   'foreign_key': True,
   'is_view': False},
  'filter_config': {'age': {'type': 'real',
    'lb': 19.392170673169762,
    'ub': 70.60807913947076},
   'sex': {'type': 'bin', 'cat_list': ['F', 'M'], 'n_total_cats': 2},
   'eth': {'type': 'ldc',
    'cat_list': ['e0', 'e1', 'e2', 'e3', 'e4', 'e5'],
    'n_total_cats': 6},
   'bin_ldc': {'type': 'ldc', 'cat_list': ['A', 'B'], 'n_total_cats': 3},
   'job': {'type': 'hdc',
    'cat_list': ['j000',
     'j001',
     'j002',
     'j003',
     'j004',
     'j005',
     'j006',
     'j008',
     'j007',
     'j009',
     'j010',
     'j012',
     'j011',
     'j015',
     'j013',
     'j016',
     'j018',
     'j017',
     'j014',
     'j019',
     'j020',
     'j026',
     'j023',
     'j021',
     'j024',
     'j022',
     'j025',
     'j02

In [11]:
k = Hopper.dbms(verbose=verbose)

for fvm_save_dict in fvm_save_dicts:
    #_tc = Hopper.table_config.load_save_dict(fvm_save_dict["tc"])
    _tc = Hopper.table_config(**fvm_save_dict["tc"])
    print(vars(_tc))
    k._create_fvm(_tc)

DROP TABLE IF EXISTS i_ids;

CREATE TABLE i_ids (
	i_id TEXT NOT NULL,
	PRIMARY KEY (i_id)
);

DROP TABLE IF EXISTS i_windows;

CREATE TABLE i_windows (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_partitions;

CREATE TABLE i_partitions (
	i_id TEXT NOT NULL,
	partition TEXT NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_relcal;

CREATE TABLE i_relcal (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL
);

{'name': 'characteristics_0', 'feature_names': ['age', 'sex', 'eth', 'bin_ldc', 'job'], 'feature_types': ['real', 'bin', 'ldc', 'ldc', 'hdc'], 'has_times': False, 'primary_key': True, 'foreign_key': True, 'is_view': False, 'info': {'tn': 'characteristics_0', 'pk_cn': 'i_id', 'st_cn': 'i_st', 'et_cn': 'i_et', 'mn_tn': 'i_ids', 'wn_tn': 'i_windows', 'pr_tn': 'i_partitions', 'rc_tn': 'i_relcal'}}
DROP

In [12]:
table_name_fp_dict = {"characteristics_0": "data/characteristics_0.csv",
                      "samples_0": "data/samples_0.csv",
                      "samples_1": "data/samples_1.csv",
                      "samples_2": "data/samples_2.csv",
                      "samples_3": "data/samples_3.csv",
                      "samples_4": "data/samples_4.csv",
                     }

for table_name, data_fp in table_name_fp_dict.items():
    fvm = k.fvm_lookup[table_name]
    k._csv_to_fvm(fvm, data_fp, delimiter=',')

INSERT OR IGNORE INTO i_ids (i_id) VALUES (?);

INSERT  INTO characteristics_0 (i_id, age, sex, eth, bin_ldc, job) VALUES (?, ?, ?, ?, ?, ?);

CREATE INDEX i_index_characteristics_0_i_id ON characteristics_0(i_id);

CREATE INDEX i_index_characteristics_0_age ON characteristics_0(age);

CREATE INDEX i_index_characteristics_0_sex ON characteristics_0(sex);

CREATE INDEX i_index_characteristics_0_eth ON characteristics_0(eth);

CREATE INDEX i_index_characteristics_0_bin_ldc ON characteristics_0(bin_ldc);

CREATE INDEX i_index_characteristics_0_job ON characteristics_0(job);

10000 rows loaded

INSERT OR IGNORE INTO i_ids (i_id) VALUES (?);

INSERT  INTO samples_0 (i_id, i_st, i_et, SBP, DBP, type) VALUES (?, ?, ?, ?, ?, ?);

CREATE INDEX i_index_samples_0_i_id ON samples_0(i_id);

CREATE INDEX i_index_samples_0_i_st ON samples_0(i_st);

CREATE INDEX i_index_samples_0_i_et ON samples_0(i_et);

CREATE INDEX i_index_samples_0_SBP ON samples_0(SBP);

CREATE INDEX i_index_samples_0_DBP ON samp

In [13]:
after_first=None
before_last=None
default_first=None
default_last=None
fit_normalization_via_sql_qds=False


if not k.windows:
    k.set_windows(data=[],
                  after_first=after_first,
                  before_last=before_last,
                  default_first=default_first,
                  default_last=default_last)
    
if not k.partitions:
    k.set_partitions(data=[])
    
k.create_windows_views()
k.create_partition_views_prior_to_filter()

#self.fit_filter() -> set filter
for fvm_save_dict in fvm_save_dicts:
    fvm_name = fvm_save_dict["tc"]["name"]
    fvm_filter_config = fvm_save_dict["filter_config"]
    #print(fvm_name, fvm_filter_config)
    
    fvm = k.fvm_lookup[fvm_name]
    fvm.filter_config = fvm_filter_config

k.set_relcal()
k.fil_agg()

k.create_partition_views_prior_to_normalization() 

#self.fit_normalization(via_sql_qds=fit_normalization_via_sql_qds) -> set nrm
for fvm_save_dict in fvm_save_dicts:
    fvm_name = fvm_save_dict["tc"]["name"]
    fvm_nrm_config = fvm_save_dict["nrm_config"]
    #print(fvm_name, fvm_filter_config)
    
    fvm = k.fvm_lookup[fvm_name]
    fvm.nrm_config = fvm_nrm_config
    
k.normalize()
k.create_final_partition_views()

INSERT  INTO i_windows (i_id, i_st, i_et) VALUES (?, ?, ?);

gen_range_table_sql called: af=None, bf=None
INSERT OR IGNORE INTO i_windows 
SELECT * FROM (
	SELECT i_id, MIN(i_st) AS i_st, MAX(i_et) AS i_et
	FROM (
	    SELECT i_id, i_st, i_et
		FROM samples_0
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_1
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_2
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_3
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_4
		
	)
	GROUP BY i_id
);

INSERT  INTO i_partitions (i_id, partition) VALUES (?, ?);

INSERT OR IGNORE INTO i_partitions 
SELECT * FROM (
	SELECT i_id, CASE 
		 WHEN r <= 0.8 THEN "train" 
		 WHEN r <= 0.9 THEN "dev" 
		 WHEN r <= 1.0 THEN "test" 
		 ELSE "test" 
	END
	FROM (SELECT i_id, ABS(RANDOM())/(9223372036854775807.0) AS r FROM i_ids)
);

CREATE VIEW win_samples_0 AS
    SELECT samples_0.i_id,
	MAX(samples_0.i_st, i_windows.i_st) AS i_st,
	MIN(samples_0.i_et, i_windows.i_et) AS i_et,
	SBP,
	DBP,
	type
	FROM (
	    sample

In [14]:
tfp_h = Prepper.tf_prepper(h)
tfp_h.features

['i_id',
 'i_duration',
 'characteristics_0/age',
 'characteristics_0/sex',
 'characteristics_0/eth_e0',
 'characteristics_0/eth_e1',
 'characteristics_0/eth_e2',
 'characteristics_0/eth_e3',
 'characteristics_0/eth_e4',
 'characteristics_0/eth_e5',
 'characteristics_0/eth__OTHER_',
 'characteristics_0/bin_ldc_A',
 'characteristics_0/bin_ldc_B',
 'characteristics_0/bin_ldc__OTHER_',
 'characteristics_0/job',
 'samples_0/avg_SBP',
 'samples_0/min_SBP',
 'samples_0/max_SBP',
 'samples_0/avg_DBP',
 'samples_0/min_DBP',
 'samples_0/max_DBP',
 'samples_0/avg_type_c0',
 'samples_0/avg_type_c1',
 'samples_0/avg_type_c2',
 'samples_0/avg_type_c3',
 'samples_0/avg_type_c4',
 'samples_0/avg_type__OTHER_',
 'samples_0/count',
 'samples_1/avg_tmps',
 'samples_1/min_tmps',
 'samples_1/max_tmps',
 'samples_1/avg_hrs',
 'samples_1/min_hrs',
 'samples_1/max_hrs',
 'samples_1/avg_sbps',
 'samples_1/min_sbps',
 'samples_1/max_sbps',
 'samples_1/avg_rrs',
 'samples_1/min_rrs',
 'samples_1/max_rrs',
 'sam

In [15]:
tfp_k = Prepper.tf_prepper(h)
tfp_k.features

['i_id',
 'i_duration',
 'characteristics_0/age',
 'characteristics_0/sex',
 'characteristics_0/eth_e0',
 'characteristics_0/eth_e1',
 'characteristics_0/eth_e2',
 'characteristics_0/eth_e3',
 'characteristics_0/eth_e4',
 'characteristics_0/eth_e5',
 'characteristics_0/eth__OTHER_',
 'characteristics_0/bin_ldc_A',
 'characteristics_0/bin_ldc_B',
 'characteristics_0/bin_ldc__OTHER_',
 'characteristics_0/job',
 'samples_0/avg_SBP',
 'samples_0/min_SBP',
 'samples_0/max_SBP',
 'samples_0/avg_DBP',
 'samples_0/min_DBP',
 'samples_0/max_DBP',
 'samples_0/avg_type_c0',
 'samples_0/avg_type_c1',
 'samples_0/avg_type_c2',
 'samples_0/avg_type_c3',
 'samples_0/avg_type_c4',
 'samples_0/avg_type__OTHER_',
 'samples_0/count',
 'samples_1/avg_tmps',
 'samples_1/min_tmps',
 'samples_1/max_tmps',
 'samples_1/avg_hrs',
 'samples_1/min_hrs',
 'samples_1/max_hrs',
 'samples_1/avg_sbps',
 'samples_1/min_sbps',
 'samples_1/max_sbps',
 'samples_1/avg_rrs',
 'samples_1/min_rrs',
 'samples_1/max_rrs',
 'sam

In [16]:
for _i, _j in zip(tfp_h.features, tfp_k.features):
    if _i != _j:
        print(_i, _j)

In [17]:
break

SyntaxError: 'break' outside loop (<ipython-input-17-6aaf1f276005>, line 4)

## Window, Filter, Transform, Aggregate, Normalize Data

In [ ]:
#h.dew_it(after_first=35)
h.dew_it(fit_normalization_via_sql_qds=False,
         default_first=0,
         default_last=5)

# Prepper 

In [ ]:
tfp = Prepper.tf_prepper(h)
tfp.features

## Fit Prepper to Desired Task

In [ ]:
tfp.fit(offsets=[0, 1, 2, 4, 5], label_fns=["samples_3/avg_outcome", ], partition="train")

## Make Models

In [ ]:
models = {}

In [ ]:
d0 = tf.keras.layers.Dense(units=32, name="encode")
r0 = tf.keras.layers.LSTM(units=32,return_sequences=True, name="RNN_0")
r1 = tf.keras.layers.LSTM(units=16,return_sequences=True, name="RNN_1")

models["all"] = tfp.build_model(middle_layer_list=[d0, r0, r1])
models["all"].compile(loss="binary_crossentropy")
models["all"].summary()

In [ ]:
d0 = tf.keras.layers.Dense(units=32, name="encode")
r0 = tf.keras.layers.LSTM(units=32,return_sequences=True, name="RNN_0")
r1 = tf.keras.layers.LSTM(units=16,return_sequences=True, name="RNN_1")

models["noWS"] = tfp.build_model(middle_layer_list=[d0, r0, r1], ignore_fns=['samples_4/avg_q', 'samples_4/count'])
models["noWS"].compile(loss="binary_crossentropy")
models["noWS"].summary()

## Make TensorFlow Dataset

In [ ]:
ds = tfp.transform_to_ds()

# Train Model

In [ ]:
models["all"].fit(ds["train"], epochs=1, validation_data=ds["dev"])
#final_model.fit(ds["test"], epochs=1)

In [ ]:
models["noWS"].fit(ds["train"], epochs=1, validation_data=ds["dev"])
#final_wo_model.fit(ds["test"], epochs=1)

# Examine Individual Trajectories

In [ ]:
for idx in ["00000", "00001"]:
    e = Prepper.entity(idx, tfp)
    e.plot()

In [ ]:
for idx in ["00000", "00001"]:
    e = Prepper.entity(idx, tfp)
    print(idx)
    for model_name, model in models.items():
        print(model_name)
        e.predict(model)
        e.plot()

# Operating Characteristics

In [ ]:
partitions = ["train", "dev", "test"]
pop_res = {partition: {} for partition in partitions}

for partition in partitions:
    for model_name, model in models.items():
        print(partition, model_name)
        pop_res[partition][model_name] = \
        Prepper.population(ds[partition], tfp, model)

# Train

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["train"][model_name].roc_curves()
    pop_res["train"][model_name].pr_curves()
    pop_res["train"][model_name].calibration_curves()
    pop_res["train"][model_name].calibration_curves(n_bins=2)
    pop_res["train"][model_name].calibration_curves(quantile=True)
    plt.show()

# Dev

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["dev"][model_name].roc_curves()
    pop_res["dev"][model_name].pr_curves()
    pop_res["dev"][model_name].calibration_curves()
    pop_res["dev"][model_name].calibration_curves(quantile=True)
    plt.show()

# Test

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["test"][model_name].roc_curves()
    pop_res["test"][model_name].pr_curves()
    pop_res["test"][model_name].calibration_curves()
    pop_res["test"][model_name].calibration_curves(quantile=True)
    plt.show()

# Examine Individual Trajectories

In [ ]:
e = Prepper.entity("00001", tfp)
e.print_labels()
e.plot()
e.predict(models["all"])
e.print_labels()
e.plot()

# Additional Feature Development

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["test"][model_name].roc_curves()
    pop_res["test"][model_name].pr_curves()
    pop_res["test"][model_name].calibration_curves()
    pop_res["test"][model_name].calibration_curves(quantile=True)
    plt.show()

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["test"][model_name].calc_weights(lambda x: 1/x)
    pop_res["test"][model_name].roc_curves()
    pop_res["test"][model_name].pr_curves()
    pop_res["test"][model_name].calibration_curves()
    pop_res["test"][model_name].calibration_curves(quantile=True)
    plt.show()

In [ ]:
for model_name, model in models.items():
    print(model_name)
    pop_res["test"][model_name].calc_weights(lambda x: math.log(x)/x)
    pop_res["test"][model_name].roc_curves()
    pop_res["test"][model_name].pr_curves()
    pop_res["test"][model_name].calibration_curves()
    pop_res["test"][model_name].calibration_curves(quantile=True)
    plt.show()

In [ ]:
len(pop_res["test"]["all"].pred["samples_3/avg_outcome"][0])

In [ ]:
len(pop_res["test"]["all"].weights["samples_3/avg_outcome"][0])

In [ ]:
idx="00000"
s_X, s_Y = tfp.get_specific_XY(idx)

In [ ]:
s_Y_h = models["all"].predict(s_X)

for i, lb in enumerate(tfp.label_fns):
    print(s_Y_h[i].shape, s_Y[lb].shape)

In [ ]:
np.set_printoptions(precision=3)
for fn, v in s_X.items():
    print(fn, "\n", np.squeeze(np.array(v)))

In [ ]:
for i in ds["train"].take(1):
    x, y = i
    for k, v in x.items():
        print(k, '\n', v.shape, '\n')
    
    for i, _y in enumerate(y):
        print("y[%s]" %(i), '\n', _y.shape, '\n')

print("Avg y value: ", tf.reduce_mean(y).numpy())